In [1]:
import numpy as np
import pandas as pd
from IPython.display import display 
import awoc
#pip install deep-translator
from deep_translator import GoogleTranslator
#pip install folium
import folium

### Аэропорты (взлеты/посадки)

Загружаем данные из Интернета

Присваиваем колонкам соответсвующие названия с сайта

Каждой стране присваиваем континент (добавляем новый столбец)

In [2]:
url_airport = 'https://raw.githubusercontent.com/jpatokal/openflights/master/data/airports.dat'
url_routes = 'https://raw.githubusercontent.com/jpatokal/openflights/master/data/routes.dat'

col_air = ['Airport ID', 'Name', 'City', 'Country','IATA', 'ICAO','Latitude', 'Longitude','Altitude','Timezone',
    'DST', 'TZ', 'Type', 'SOURCE']
airport = pd.read_csv(url_airport,names=col_air)
airport = airport.replace('\\N',np.nan)

col_routes = ['Airline', 'Airline ID', 'Source airport', 'Source airport ID', 'Destination airport',
              'Destination airport ID', 'Codeshare','Stops', 'Equipment']
routes = pd.read_csv(url_routes,names=col_routes)

my_world = awoc.AWOC()
def continent_from_country(name):
    try:
        continent = my_world.get_country_continent_name(name)
    except NameError:
        continent = np.nan
    return continent

df_air = airport.copy()
df_air['Continent'] = df_air['Country'].map(continent_from_country)

df_air[df_air['Continent'].isna()]['Country'].unique()
df_continent = {
    "Cote d'Ivoire":'Africa',
    'Congo (Brazzaville)':'Africa',
    'Congo (Kinshasa)':'Africa',
    'West Bank':'Asia',
    'Midway Islands':'Oceania',
    'French Guiana':'South America',
    'Martinique':'North America',
    'Guadeloupe':'North America',
    'Virgin Islands':'North America',
    'Burma':'Asia',
    'Norfolk Island':'Oceania',
    'Johnston Atoll':'Oceania',
    'Cocos (Keeling) Islands':'Asia',
    'Svalbard':'Europe',
    'Wake Island':'Oceania'
}
df_air['Continent'] = df_air['Continent'].fillna(df_air['Country'].map(df_continent))
df_air.head()

,Airport ID,Name,City,Country,IATA,ICAO,Latitude,Longitude,Altitude,Timezone,DST,TZ,Type,SOURCE,Continent
0,1,Goroka Airport,Goroka,Papua New Guinea,GKA,AYGA,-6.081690,145.391998,5282,10,U,Pacific/Port_Moresby,airport,OurAirports,Oceania
1,2,Madang Airport,Madang,Papua New Guinea,MAG,AYMD,-5.207080,145.789001,20,10,U,Pacific/Port_Moresby,airport,OurAirports,Oceania
2,3,Mount Hagen Kagamuga Airport,Mount Hagen,Papua New Guinea,HGU,AYMH,-5.826790,144.296005,5388,10,U,Pacific/Port_Moresby,airport,OurAirports,Oceania
3,4,Nadzab Airport,Nadzab,Papua New Guinea,LAE,AYNZ,-6.569803,146.725977,239,10,U,Pacific/Port_Moresby,airport,OurAirports,Oceania
4,5,Port Moresby Jacksons International Airport,Port Moresby,Papua New Guinea,POM,AYPY,-9.443380,147.220001,146,10,U,Pacific/Port_Moresby,airport,OurAirports,Oceania


### Часть 1

Все дальнейшие действия будут выполняться для Франции

1. Отобираем все страны, которые находятся на моем континенте и выводим количество найденных стран.

In [3]:
my_country = 'France'
my_continent = df_air[df_air['Country'] == my_country]['Continent'].unique()[0]
kolvo_cou_in_con = len(df_air[df_air['Continent'] == my_continent]['Country'].unique())
print('Моя страна:', my_country,' находится на континенте:', my_continent,'\nНа этом континенте:', 
      kolvo_cou_in_con,' стран.')

Моя страна: France  находится на континенте: Europe 
На этом континенте: 46  стран.


2. Для каждой страны отобираем по 3 крупнейших аэропорта (только IATA) по сумме вылетов и прилетов в данный аэропорт (взлет/посадки).


In [4]:
airport_clean = df_air.dropna(subset=['IATA']).copy()
routes_source = routes[['Airline ID','Source airport']].groupby(['Source airport']).count()
routes_source.index.name = 'IATA'
routes_source.columns = ['Source']
routes_destin = routes[['Airline ID','Destination airport']].groupby('Destination airport').count()
routes_destin.index.name = 'IATA'
routes_destin.columns = ['Destination']
routes_amount = routes_source.join(routes_destin)
routes_amount['SUM'] = routes_amount.sum(axis=1)
sum_airport = pd.merge(airport_clean, routes_amount['SUM'], left_on = 'IATA', right_index = True)

sum_airport_larg3 = sum_airport[sum_airport['Continent'] == my_continent].groupby('Country',as_index = False).apply(
    lambda x: x.nlargest(3,'SUM'))[['Country','Name','SUM']]
sum_airport_larg3_res = sum_airport_larg3.groupby(['Country','Name'],sort=False).sum()
sum_airport_larg3_res

SUM
Country        Name                                              
Albania        Tirana International Airport Mother Teresa    72.0
Austria        Vienna International Airport                 616.0
               Salzburg Airport                              69.0
               Graz Airport                                  38.0
Belarus        Minsk National Airport                       108.0
...                                                           ...
Ukraine        Kiev Zhuliany International Airport           44.0
               Odessa International Airport                  37.0
United Kingdom London Heathrow Airport                     1051.0
               London Gatwick Airport                       708.0
               Manchester Airport                           627.0

[106 rows x 1 columns]

3. Таблица из предыдушего задания сортируем по названиям стран со вторым наибольшим аэропортом, а внутри страны от наибольшего к наименьшему (при отсутствии второго аэропорта, берется первый).

In [5]:
keys = sum_airport_larg3_res.index.get_level_values(0).unique()
values = []
for x in keys:
    if len(sum_airport_larg3_res.loc[sum_airport_larg3_res.index.get_level_values(0) == x]) != 2:
        values.append(sum_airport_larg3_res.loc[sum_airport_larg3_res.index.get_level_values(0) == x]['SUM'].median())
    else:
        values.append(sum_airport_larg3_res.loc[sum_airport_larg3_res.index.get_level_values(0) == x]['SUM'].iloc[0])
dict1 = {}
for i in range(len(keys)):
    dict1[keys[i]] = values[i]
sorted_dict = {}
sorted_keys = sorted(dict1, key=dict1.get,reverse = True)
for i in sorted_keys:
    sorted_dict[i] = dict1[i]
    
sum_airport_larg3_res.loc[sorted_dict.keys()]

SUM
Country                Name                                    
Germany                Frankfurt am Main Airport          990.0
                       Munich Airport                     728.0
                       Düsseldorf Airport                 570.0
United Kingdom         London Heathrow Airport           1051.0
                       London Gatwick Airport             708.0
...                                                         ...
Bosnia and Herzegovina Banja Luka International Airport     6.0
                       Tuzla International Airport          6.0
Estonia                Lennart Meri Tallinn Airport        76.0
                       Tartu Airport                        4.0
                       Kuressaare Airport                   2.0

[106 rows x 1 columns]

4. Аналогично пункт 2, но названия стран отображаются на русском языке.

In [6]:
sum_airport_larg3_rus = sum_airport_larg3.groupby(['Country','Name'],sort=False, as_index = False).sum()
m = []
for x in sum_airport_larg3_rus['Country']:
    m.append(GoogleTranslator(source='auto', target='ru').translate(x))
sum_airport_larg3_rus['Country'] = m
sum_airport_larg3_rus.groupby(['Country','Name'],sort=False).sum()

SUM
Country        Name                                              
Албания        Tirana International Airport Mother Teresa    72.0
Австрия        Vienna International Airport                 616.0
               Salzburg Airport                              69.0
               Graz Airport                                  38.0
Беларусь       Minsk National Airport                       108.0
...                                                           ...
Украина        Kiev Zhuliany International Airport           44.0
               Odessa International Airport                  37.0
Великобритания London Heathrow Airport                     1051.0
               London Gatwick Airport                       708.0
               Manchester Airport                           627.0

[106 rows x 1 columns]

### Часть 2

Визуализация 

1.	На карте обозначаем:
- все аэропорты моей страны цветом 1; 
- цветом 2 аэропорты с кодом IATA;
- цветом 3 аэропорты, из которых осуществляются международные авиаперелеты (вылеты).


In [7]:
all_air = df_air[df_air['Country'] == my_country]
iata_air = airport_clean[airport_clean['Country'] == my_country]
routes_my_coun = routes[routes['Source airport'].isin(iata_air["IATA"])]
routes_my_coun = routes_my_coun[routes_my_coun['Destination airport'].isin(airport['IATA'])]
routes_my_coun = routes_my_coun[~routes_my_coun['Destination airport'].isin(iata_air["IATA"])]
routes_source = routes_my_coun[['Airline ID','Source airport']].groupby(['Source airport']).count()
routes_source.index.name = 'IATA'
routes_source.columns = ['Source']
source_air = pd.merge(airport_clean, routes_source, left_on = 'IATA', right_index = True)

feature_group1 = folium.FeatureGroup(name='all airports')
feature_group2 = folium.FeatureGroup(name='airports with IATA')
feature_group3 = folium.FeatureGroup(name='airports with international sources')
all_airports = folium.Map(location=[47.5, 3],zoom_start=5.5)
for x in range(len(all_air)):
    lat = all_air.iloc[x]['Latitude']
    lon = all_air.iloc[x]['Longitude']
    name = all_air.iloc[x]['Name']
    folium.Marker(
        [lat,lon], tooltip=name, icon=folium.Icon(icon='plane', color='orange')).add_to(feature_group1)
for x in range(len(iata_air)):
    lat = iata_air.iloc[x]['Latitude']
    lon = iata_air.iloc[x]['Longitude']
    name = iata_air.iloc[x]['Name']
    folium.Marker(
        [lat,lon], tooltip=name, icon=folium.Icon(icon='plane', color='blue')).add_to(feature_group2)
for x in range(len(source_air)):
    lat = source_air.iloc[x]['Latitude']
    lon = source_air.iloc[x]['Longitude']
    name = source_air.iloc[x]['Name']
    folium.Marker(
        [lat,lon], tooltip=name, icon=folium.Icon(icon='plane', color='red')).add_to(feature_group3)
all_airports.add_child(feature_group1)
all_airports.add_child(feature_group2)
all_airports.add_child(feature_group3)
all_airports.add_child(folium.map.LayerControl())

2.	На карте строим международную транспортную сеть по вылетам из моей страны.

In [8]:
routes_source = routes_my_coun[['Airline ID','Source airport','Destination airport']].groupby([
    'Source airport','Destination airport']).count()
routes_source.columns = ['Source']

all_airports = folium.Map(location=[47.5, 3],zoom_start=2)
for x in range(len(routes_source)):
    s = airport[airport['IATA'] == routes_source.index.get_level_values(0)[x]].iloc[0]
    d = airport[airport['IATA'] == routes_source.index.get_level_values(1)[x]].iloc[0]
    lats = s['Latitude']
    lons = s['Longitude']
    names = s['Name']
    latd = d['Latitude']
    lond = d['Longitude']
    named = d['Name']
    folium.Marker(
        [lats,lons], tooltip=names, icon=folium.Icon(icon='plane', color='red')).add_to(all_airports)
    folium.Marker(
        [latd,lond], tooltip=named, icon=folium.Icon(icon='plane', color='darkgreen')).add_to(all_airports)
    w = int(routes_source['Source'][x])
    folium.PolyLine(locations = [[lats,lons],[latd,lond]], weight = w,color = 'blue').add_to(all_airports)

all_airports